In [42]:
import os
import requests
from dotenv import load_dotenv
from IPython.display import display, HTML
import re

# Load environment variables from the .env file
load_dotenv()

# Get the API key from environment variables
api_key = os.getenv("OPENAI_API_KEY")


In [10]:
import requests

def generate_openai_text(api_key, model, messages, temperature=1.0, max_tokens=2000, top_p=1.0, frequency_penalty=0.0, presence_penalty=0.0):
    """
    Generate text using OpenAI's API.
    
    Parameters:
    - api_key (str): The API key for OpenAI.
    - model (str): The model to use for text generation.
    - messages (list): A list of messages to pass to the API in a conversation format.
    - temperature (float): Controls randomness in the output (0-1).
    - max_tokens (int): The maximum number of tokens to generate in the completion.
    - top_p (float): Controls the diversity via nucleus sampling (0-1).
    - frequency_penalty (float): Controls the repetition of words (0-1).
    - presence_penalty (float): Controls the introduction of new topics (0-1).
    
    Returns:
    - response (dict): The API response as a dictionary.
    """
    url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    data = {
        "model": model,
        "messages": messages,
        "temperature": temperature,
        "max_tokens": max_tokens,
        "top_p": top_p,
        "frequency_penalty": frequency_penalty,
        "presence_penalty": presence_penalty
    }

    try:
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None


In [11]:
def format_openai_response(response):
    """
    Formats the response from OpenAI API to display only the assistant's message.
    
    Parameters:
    - response (dict): The API response as a dictionary.
    
    Returns:
    - formatted_text (str): A formatted string with Markdown for the assistant's message.
    """
    if response and "choices" in response:
        assistant_message = response["choices"][0]["message"]["content"]
        formatted_text = f"**Assistant:**\n\n{assistant_message}\n"
        return formatted_text
    else:
        return "No valid response received."


In [12]:
class ChatBot:
    def __init__(self, api_key, model="gpt-4o", temperature=1.0, max_tokens=256, top_p=1.0, frequency_penalty=0.0, presence_penalty=0.0):
        """
        Initialize the ChatBot with API key and parameters.
        
        Parameters:
        - api_key (str): The API key for OpenAI.
        - model (str): The model to use for text generation.
        - temperature (float): Controls randomness in the output (0-1).
        - max_tokens (int): The maximum number of tokens to generate in the completion.
        - top_p (float): Controls the diversity via nucleus sampling (0-1).
        - frequency_penalty (float): Controls the repetition of words (0-1).
        - presence_penalty (float): Controls the introduction of new topics (0-1).
        """
        self.api_key = api_key
        self.model = model
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.top_p = top_p
        self.frequency_penalty = frequency_penalty
        self.presence_penalty = presence_penalty
        self.conversation_history = []

    def add_message(self, role, content):
        """
        Add a message to the conversation history.
        
        Parameters:
        - role (str): The role of the sender ("user" or "assistant").
        - content (str): The content of the message.
        """
        self.conversation_history.append({"role": role, "content": content})

    def get_response(self, user_input):
        """
        Get a response from the OpenAI API based on the user input.
        
        Parameters:
        - user_input (str): The user's input message.
        
        Returns:
        - assistant_reply (str): The assistant's generated response.
        """
        # Add user input to conversation history
        self.add_message("user", user_input)

        # Call the OpenAI API to get a response
        response = generate_openai_text(
            self.api_key,
            self.model,
            self.conversation_history,
            temperature=self.temperature,
            max_tokens=self.max_tokens,
            top_p=self.top_p,
            frequency_penalty=self.frequency_penalty,
            presence_penalty=self.presence_penalty
        )

        if response:
            # Extract the assistant's reply
            assistant_reply = response["choices"][0]["message"]["content"]
            # Add assistant's reply to conversation history
            self.add_message("assistant", assistant_reply)
            return assistant_reply
        else:
            return "Sorry, I couldn't generate a response."



In [13]:
model = "gpt-4o-mini"  # Replace with your desired model
messages = [
    {"role": "user", "content": "Can you explain the concept of supply and demand in economics?"}
]

response = generate_openai_text(api_key, model, messages)
formatted_response = format_openai_response(response)
print(formatted_response)


**Assistant:**

Certainly! Supply and demand are fundamental concepts in economics that describe how markets function and determine prices and quantity of goods and services exchanged in an economy.

### Demand
Demand refers to the quantity of a good or service that consumers are willing and able to purchase at various prices, over a given time period. The relationship between price and quantity demanded is typically inverse: as the price of a good decreases, the quantity demanded usually increases, and vice versa. This relationship is represented graphically by a downward-sloping demand curve.

Several factors can affect demand, including:
- **Price of the good**: Generally, lower prices lead to higher demand.
- **Consumer income**: An increase in income can lead to an increase in demand, especially for normal goods.
- **Preferences and tastes**: Changes in consumer preferences can increase or decrease demand.
- **Prices of related goods**: This includes substitutes (which can have an

In [16]:
chatbot = ChatBot(api_key)

In [17]:
user_input = "Hello"
print(f"User: {user_input}")
response = chatbot.get_response(user_input)
print(f"Assistant: {response}")

User: Hello
Assistant: Hello! How can I assist you today?


In [18]:
user_input = "What are some effective strategies for personal growth and self-improvement?"
print(f"User: {user_input}")
response = chatbot.get_response(user_input)
print(f"Assistant: {response}")

User: What are some effective strategies for personal growth and self-improvement?
Assistant: Personal growth and self-improvement are ongoing processes that can be approached from various angles. Here are some effective strategies to consider:

1. **Set Clear Goals**: 
   - Create specific, measurable, attainable, relevant, and time-bound (SMART) goals. 
   - Break down long-term goals into smaller, manageable tasks.

2. **Develop a Routine**: 
   - Consistency is key. Establish daily and weekly routines that support your goals.
   - Incorporate habits that promote productivity and well-being.

3. **Lifelong Learning**: 
   - Continuously seek out new knowledge and skills. This can be through formal education, online courses, reading books, or attending workshops.
   - Stay curious and open-minded.

4. **Self-Reflection**: 
   - Regularly assess your progress and areas for improvement. Journaling can be a helpful tool for self-reflection.
   - Understand your strengths and weaknesses.

In [19]:
user_input = "Which of these strategies do you think is the most important and why?"
print(f"User: {user_input}")
response = chatbot.get_response(user_input)
print(f"Assistant: {response}")

User: Which of these strategies do you think is the most important and why?
Assistant: The importance of each strategy can vary depending on individual circumstances and goals, but if I had to choose one that often serves as a foundational element for many aspects of personal growth, it would be **Setting Clear Goals**. Here’s why:

1. **Direction and Focus**: Clear goals provide a roadmap for where you want to go. They help you focus your efforts and resources on what's most important, preventing you from getting sidetracked or overwhelmed by too many possibilities.

2. **Motivation and Commitment**: Knowing exactly what you're working towards can be incredibly motivating. Goals serve as a constant reminder of why you're putting in the effort, which is essential for maintaining commitment over time.

3. **Measurement and Accountability**: Having specific goals allows you to measure your progress objectively. This provides a sense of accomplishment as you reach milestones and helps you